<a href="https://colab.research.google.com/github/bthodla/danano/blob/master/prj4/wrangle_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Gathering data

1.   Load the "twitter-archive-enhanced.csv" into a dataframe titled "tweets_df"
2.   Load the "image_predictions.tsv" (the Tweet image predictions) hosted on Udacity servers programmatically using the "requests" library (URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv) and store it in a dataframe titled "img_pred_df".
3. Using tweet ids from (1) above and the Twitter API, load information such as the following and store them in a JSON file titled "tweet_json.txt":


*   Retweet Count
*   Favorite Count
*   Media URL


4.   Read this file line by line into a data frame titled "tweets_mini_df"

### Assessing data

Key Points

* For each dataframe created as per steps in the "Gathering Data" section, display and view the following information:

    * DataFrame.info(): to list the columns of the dataframe
    * DataFrame.Head() or DataFrame.sample(5): to view a sample of the data
    * DataFrame.shape: to list the number of rows (and columns) in the dataframe
    

### Cleaning data

#### 1. twitter-archive-enhanced.csv
##### 1.1 Data Quality Issues:

1. There are 181 rows in this dataset that are retweets. I have identified them based non-null values in the "retweeted_status_id" column. I am removing these from the dataset as we only want original tweets (1.1.1)

2. Remove the following columns from the dataset. I don't think that we need these columns for our analysis (1.1.2)

    *  in_reply_to_status_id
    *  in_reply_to_user_id
    *  retweeted_status_id
    *  retweeted_status_user_id
    *  retweeted_status_timestamp
    *  expanded_urls

3. The "timestamp" column is represented as a string and I am converting this to datetime (1.1.3)
4. Drop all tweets with timestamps greater than 8/1/2017 as per the project requirements (1.1.4)
5. Dog stages are not fully recorded and many of them contain null values or values labeled as "None". We need to scan the tweet text and see if we can extract these values for each row (1.1.5)
6. *** Not Addressed *** There are 745 rows in this dataset where the name is marked as "None". I am not addressing this issue right now since I don't have sufficient data to obtain this information (1.1.6)

##### 1.2 Tidiness Issues:
1. Dog stages are represented in separate columns as a sparse-array matrix which is wasteful and also not scalable in case we identify new stages. These need to be converted to a single column titled "dog_stage". If they are not mutually exclusive (meaning, if there are multiple tweets about the same dog at different points in time where their stages change), we can put them into a separate dataframe for normalization purposes (1.2.1)


#### 2. image-predictions.tsv

##### 2.1 Tidiness Issues:

1. In the case of this dataset, we will deal with the tidiness issues first before we deal with data quality issues. There is a specific reason for it - there are three breed predictions in this dataset with a percentage of confidence against each and a boolean flag indicating whether the image prediction is actually canine or not. That makes it a total of 9 columns. I would like to reduce them to three columns to begin with: breed, prediction_confidence, is_canine. Addressing this tidiness issue will make it easier to fix the data quality issues identified below. We will begin by separating breed predictions to their own dataframe so that we can work on the data quality issues later (2.1.1)

##### 2.2 Data Quality Issues:

1. Once we address the tidiness issue above and spin off the breed predictions to their own dataframe, we will eliminate all predictions where the "is_canine" flag is false. We will then pick the prediction with the higher percentage of confidence and retain them as the only breed predictions (assuming that each dog can be classified into a single breed) and drop the ones with lower prediction confidence scores (2.2.1)
2. We can then merge this dataset back to the original dataset (2.2.2)
3. After merging, we will drop the following columns as they are no longer needed (2.2.3): 
    *  jpg_url (we have obtained the media_url using the Twitter API and we will retain that in case it needs to be used)
    *  img_num (don't see how this information will be useful)
    * The columns below will be replaced by a single breed value and a percentage of confidence in that prediction
        *  p1
        *  p1_conf
        *  p1_dog
        *  p2
        *  p2_conf
        *  p2_dog
        *  p3
        *  p3_conf
        *  p3_dog


#### 3. tweet_mini_df

##### 3.1 Data Quality Issues
1. We need to drop all the tweets in this dataset that were originally marked as retweets in the "twitter-archive-enhanced" dataset as we are only interested in original tweets (3.1.1)
2. We will also drop the "text" column as we no longer need this (3.1.2)


#### 4. Tidiness Issues: combining data from all the datasets

##### 4.1 Tidiness

1. Having addressed the data quality and tidiness issues at the level of individual datasets, we can now merge the data in these datasets into one. Since the data granularity in each of these datasets is at the "tweet_id" level, we will use it to combine the data into a single dataset  (4.1.1)

##### 4.2 Data Quality

1. We will use "inner joins" to merge these datasets thus eliminating any rows for whcih no valid information is available online. For example, there are 17 tweet ids in the original "twitter-archive-enhanced" dataset for which the Twitter API returned errors and these will be eliminated during the merging process. (4.2.1)

Finnaly, we will persist this combined dataframe into a file titled  "twitter_archive_master_file.csv". In the same manner, we will also persist the separate "Dog Stages" dataframe to a file titled "dog_stage_file.csv"